In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="1"
igfold = 4
os.system('mkdir -p ./data_folds/fold_' + str(igfold))

0

In [2]:
!nvidia-smi

Fri Jan 13 13:25:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:85:00.0 Off |                    0 |
| N/A   41C    P0    77W / 163W |   1174MiB / 32510MiB |     69%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:8A:00.0 Off |                    0 |
| N/A   

In [3]:
import cudf
import glob
import gc

from tqdm import tqdm

In [4]:
lb = False
path = '../data/'
type_weight = {0:1, 1:6, 2:3}
no_files = 3
path = '../../data/'
    
df_type = cudf.DataFrame({
    'type': ['clicks', 'carts', 'orders'],
    'type_': [0, 1, 2]
})

def list_in_chunks(files, no_chunks=10):
    out = [[] for _ in range(no_chunks)]
    for i, file in enumerate(files):
        out[i%no_chunks].append(file)
    return(out)

In [5]:
# !rm -rf ./data/
!mkdir -p ./data/

In [6]:
files = sorted(
    glob.glob('../../data/' + '/train/interim/*.parquet')
)
files_split = [glob.glob('../../data/test.parquet')] + [glob.glob('./data/xgb_train_x.parquet')] + list_in_chunks(files, no_chunks=len(files)//no_files)

In [7]:
len(files_split)

45

In [8]:
import pickle

sessions = pickle.load(open('./data/sessions_eval.pickle', 'rb'))
if igfold == 0:
    sess_eval = sessions[0]+sessions[1]
elif igfold == 1:
    sess_eval = sessions[2]+sessions[3]
elif igfold == 2:
    sess_eval = sessions[4]+sessions[5]
elif igfold == 3:
    sess_eval = sessions[6]+sessions[7]
elif igfold == 4:
    sess_eval = sessions[8]+sessions[9]
print(len(sess_eval))

356746


### 3) "Clicks" Co-visitation Matrix - Time Weighted

In [9]:
import cupy

In [10]:
for ty1 in [1]:
    os.system('rm -r /tmp/data/tmp_1')
    os.system('mkdir -p /tmp/data/tmp_1/split')
    if ty1 == 0:
        ty2s = [2]
    else:
        ty2s = [2]
    for ty2 in ty2s:
        print(ty1, ty2)
        out = []
        for e, file in tqdm(enumerate(files_split)):
            df = cudf.read_parquet(file)
            df = df.merge(
                df_type,
                how='left',
                on='type'
            )
            if any([True for x in file if 'xgb_train_x.parquet' in x]):
                print(file)
                df = df[df['session'].isin(sess_eval)]
            else:
                df = df.loc[~(df['session'].isin(sess_eval))]
            if ty1 == 0 and ty2 == 1:
                df = df.sort_values(['session', 'ts'], ascending=[True, False])
                df['ts_2'] = df.groupby(['session']).ts.shift(1)
                df['diff_'] = df['ts_2']-df['ts']
                df['diff_'] = ((df['diff_']/1000)>(2*24*60*60)).fillna(0).astype('int8')
                df['new_session'] = df.groupby(['session']).diff_.cumsum()
                df.drop(['ts_2', 'diff_'], axis=1, inplace=True)
                df['session'] = df['session']*100+df['new_session']
                df.drop(['new_session'], axis=1, inplace=True)

            df['session'] = df['session'].astype('int32')
            df['aid'] = df['aid'].astype('int32')
            df.ts = (df.ts/1000).astype('int32')
            df.drop(['type'], axis=1, inplace=True)
            df = df.rename(columns={'type_': 'type'})
            
            df = df.sort_values(['session','ts'],ascending=[True,False])
            df = df.reset_index(drop=True)
            df['n'] = df.groupby('session').cumcount()
            df['n'] = df['n'].astype('int16')
            
            df_r = df[df['type']==ty1].reset_index(drop=True)
            df_l = df[df['type']==ty2].reset_index(drop=True)
            
            del df
            gc.collect()
            
            df = df_r.merge(
                df_l[['session', 'aid', 'ts', 'n']], 
                how='left',
                on='session'
            )
            df = df[~df['aid_y'].isna()]
            
            del df_r, df_l
            gc.collect()
            
            df['diff_time'] = (df['ts_x']-df['ts_y']).abs()
            df.drop(['ts_x', 'ts_y'], axis=1, inplace=True)
            df['w_time'] = 1/cupy.log(df['diff_time']+2)
            df.drop(['diff_time'], inplace=True, axis=1)

            df['diff_pos'] = (df['n_x']-df['n_y']).abs()
            df['w_pos'] = 1/cupy.log(df['diff_pos']+2)
            df.drop(['diff_pos'], inplace=True, axis=1)

            df['wgt'] = df['w_time']*df['w_pos']
            
            df = df[['session', 'aid_x', 'aid_y', 'wgt']].groupby(['session', 'aid_x', 'aid_y']).agg(['sum', 'count'])
            df = df.reset_index()
            
            df.columns = ['session', 'aid_x', 'aid_y', 'wgt_sum', 'wgt_count']
            
            df = df.groupby(['aid_x', 'aid_y']).agg({
                'wgt_sum': 'sum',
                'wgt_count': 'sum',
                'session': 'count'
            })
            df = df.reset_index()
            df.columns = ['aid_x', 'aid_y', 'wgt_sum', 'wgt_count', 'count']
            df.to_parquet('/tmp/data/tmp_1/split/split_' + str(e) + '.parquet')
            del df
            gc.collect()
        
        files_split1 = glob.glob('/tmp/data/tmp_1/split/split_*.parquet')
        df = cudf.read_parquet(files_split1[0])
        for iifile, file in enumerate(files_split1[1:]):
            print(file)
            df2 = cudf.read_parquet(file)
            df = cudf.concat([
                df,
                df2
            ])
            gc.collect()
            df = df.groupby(['aid_x','aid_y']).sum()
            df = df.reset_index()
            if ty1==0 and ty2==1:
                if iifile==30:
                    print('Filter')
                    df = df[df['count']>1]
            del df2
            gc.collect()
        
        df = df[df['count']>1]
        gc.collect()
        files_aid = sorted(glob.glob('../../data/' + '/train/interim/*.parquet')) + glob.glob('../../data/test.parquet') + glob.glob('./data/xgb_train_x.parquet')
        out = []
        for file in files_aid:
            df_tmp = cudf.read_parquet(file)
            if 'xgb_train_x.parquet' in file:
                print(file)
                df_tmp = df_tmp[df_tmp['session'].isin(sess_eval)]
            else:
                df_tmp = df_tmp[~(df_tmp['session'].isin(sess_eval))]
            out.append(df_tmp)
        df_aid = cudf.concat(out)
        del out
        gc.collect()
        
        df_aid = df_aid.merge(
            df_type,
            how='left',
            on='type'
        )
        df_aid = df_aid[df_aid['type_']==ty1]
        df_aid = df_aid.drop_duplicates(['session', 'aid'])
        df_aid = df_aid['aid'].value_counts().reset_index()
        df_aid = df_aid.reset_index()
        df_aid.columns = ['aid_', 'aid', 'count']
        df_aid.drop(['aid_'], inplace=True, axis=1)
        df_aid.columns = ['aid_x', 'count_x']
        df = df.merge(
            df_aid,
            how='left',
            on=['aid_x']
        )
        
        files_aid = sorted(glob.glob('../../data/' + '/train/interim/*.parquet')) + glob.glob('../../data/test.parquet') + glob.glob('./data/xgb_train_x.parquet')
        out = []
        for file in files_aid:
            df_tmp = cudf.read_parquet(file)
            if 'xgb_train_x.parquet' in file:
                print(file)
                df_tmp = df_tmp[df_tmp['session'].isin(sess_eval)]
            else:
                df_tmp = df_tmp[~(df_tmp['session'].isin(sess_eval))]
            out.append(df_tmp)
        df_aid = cudf.concat(out)
        del out
        gc.collect()
        df_aid = df_aid.merge(
            df_type,
            how='left',
            on='type'
        )
        df_aid = df_aid[df_aid['type_']==ty2]
        df_aid = df_aid.drop_duplicates(['session', 'aid'])
        df_aid = df_aid['aid'].value_counts().reset_index()
        df_aid = df_aid.reset_index()
        df_aid.columns = ['aid_', 'aid', 'count']
        df_aid.drop(['aid_'], inplace=True, axis=1)
        df_aid.columns = ['aid_y', 'count_y']
        df = df.merge(
            df_aid,
            how='left',
            on=['aid_y']
        )
        
        df['wgt_sum_2'] = df['wgt_sum']/(cupy.sqrt((df['count_x'].fillna(0)+1).values)*cupy.sqrt((df['count_y'].fillna(0)+1).values))
        df['wgt_prob'] = df['count']/cupy.sqrt((df['count_x'].fillna(0)+1).values)
        df['wgt_prob_2'] = df['count']/(cupy.sqrt((df['count_x'].fillna(0)+1).values)*cupy.sqrt((df['count_y'].fillna(0)+1).values))
        df[['aid_x', 'aid_y', 'wgt_sum', 'wgt_sum_2', 'wgt_prob', 'wgt_prob_2', 'count']].to_parquet(
            './data_folds/fold_' + str(igfold) + '/top_20_wt_' + str(ty1) + '_' + str(ty2) + '.parquet'

        )
        del df
        del df_aid
        gc.collect()

1 2


1it [00:01,  1.61s/it]

['./data/xgb_train_x.parquet']


45it [00:52,  1.16s/it]


/tmp/data/tmp_1/split/split_9.parquet
/tmp/data/tmp_1/split/split_36.parquet
/tmp/data/tmp_1/split/split_39.parquet
/tmp/data/tmp_1/split/split_25.parquet
/tmp/data/tmp_1/split/split_24.parquet
/tmp/data/tmp_1/split/split_6.parquet
/tmp/data/tmp_1/split/split_28.parquet
/tmp/data/tmp_1/split/split_35.parquet
/tmp/data/tmp_1/split/split_11.parquet
/tmp/data/tmp_1/split/split_14.parquet
/tmp/data/tmp_1/split/split_32.parquet
/tmp/data/tmp_1/split/split_44.parquet
/tmp/data/tmp_1/split/split_31.parquet
/tmp/data/tmp_1/split/split_42.parquet
/tmp/data/tmp_1/split/split_12.parquet
/tmp/data/tmp_1/split/split_19.parquet
/tmp/data/tmp_1/split/split_34.parquet
/tmp/data/tmp_1/split/split_23.parquet
/tmp/data/tmp_1/split/split_16.parquet
/tmp/data/tmp_1/split/split_37.parquet
/tmp/data/tmp_1/split/split_30.parquet
/tmp/data/tmp_1/split/split_22.parquet
/tmp/data/tmp_1/split/split_29.parquet
/tmp/data/tmp_1/split/split_27.parquet
/tmp/data/tmp_1/split/split_13.parquet
/tmp/data/tmp_1/split/split

### Generate Candidates

In [11]:
!nvidia-smi

Fri Jan 13 13:27:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:85:00.0 Off |                    0 |
| N/A   38C    P0    50W / 163W |   8442MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:8A:00.0 Off |                    0 |
| N/A   

In [12]:
import cudf
import numpy as np
import gc
import os

import glob

In [13]:
def get_candidates(df, co, prefix, treshhold=2, treshhold_topn=15):
    df = df.merge(
        co[co['n']<treshhold_topn],
        how='left',
        on='aid'
    )
    df.drop(['n'], axis=1, inplace=True)
    for col in ['wgt_sum', 'wgt_sum_2', 'wgt_prob', 'wgt_prob_2', 'count']:
        df[col] = df[col].fillna(0.0)
    for col in ['wgt_sum_2', 'wgt_prob', 'count']:
        df[col + '_m'] = df[col].values
        df[col + '_2'] = df[col]*df['score']
    df.drop(['aid', 'ts', 'type'], axis=1, inplace=True)
    
    agg_dict = {
        'dummy': 'sum'
    }
    for col in ['wgt_sum_2', 'wgt_prob', 'count']:
        agg_dict[col] = 'sum'
        #agg_dict[col + '_m'] = 'max'
        agg_dict[col + '_2'] = 'sum'
    
    df = df.groupby(['session', 'cand']).agg(agg_dict).reset_index()
    df.columns = ['session', 'cand'] + [prefix + '_' + x for x in df.columns if x not in ['session', 'cand']]
    return(df)

def list_in_chunks(files, no_chunks=10):
    out = [[] for _ in range(no_chunks)]
    for i, file in enumerate(files):
        out[i%no_chunks].append(file)
    return(out)

def add_log_recency_score(df):
    linear_interpolation = 0.1 + ((1-0.1) / (df['session_len']-1)) * (df['session_len']-df['rank'])
    df['score'] = (2 ** linear_interpolation - 1).fillna(1.0)
    return df

df_type = cudf.DataFrame({
    'type': ['clicks', 'carts', 'orders'],
    'type_': [0, 1, 2]
})

In [14]:
mainprefix = 'weighted'

In [15]:
for ty1 in [1]:
    if ty1 == 0:
        ty2s = [1, 2]
    else:
        ty2s = [2]
    for ty2 in ty2s:
        prefix = mainprefix + '_' + str(ty1) + '_' + str(ty2)
        for dataset in ['train', 'sub']:
            for treshhold_topn in [20]:
                print(dataset, treshhold_topn)
                os.system('mkdir -p ./data_folds/fold_' + str(igfold) + '/candidates/' + dataset + '/' + prefix + '_' + str(treshhold_topn))
                co = cudf.read_parquet('./data_folds/fold_' + str(igfold) + '/top_20_wt_' + str(ty1) + '_' + str(ty2) + '.parquet')
                co = co.sort_values(['aid_x','wgt_sum_2'],ascending=[True,False])
                co = co.reset_index(drop=True)
                co['n'] = co.groupby('aid_x').aid_y.cumcount()
                co.columns = ['aid', 'cand', 'wgt_sum', 'wgt_sum_2', 'wgt_prob', 'wgt_prob_2', 'count', 'n']
                if dataset == 'train':
                    df = cudf.read_parquet('./data/xgb_train_x.parquet')
                    df = df[df['session'].isin(sess_eval)]
                else:
                    df = cudf.read_parquet('../../data/test.parquet')
                df = df.merge(
                    df_type,
                    how='left',
                    on='type'
                )
                df.drop(['type'], axis=1, inplace=True)
                df = df.rename(columns={'type_': 'type'})
                df = df[df['type']==ty1]
                df = df.sort_values(['session', 'ts'], ascending=[True, False])
                df['dummy'] = 1
                df['rank'] = df.groupby(['session']).dummy.cumsum()
                df = df.merge(
                    df[['session']].groupby(['session']).size().reset_index().rename(columns={0: 'session_len'}),
                    how='left',
                    on=['session']
                )
                df['session_len'] = df['session_len'].astype('int16')
                df = add_log_recency_score(df)
                session_lists = list_in_chunks(df['session'].drop_duplicates().to_pandas().values.tolist(), no_chunks=10)
                out = []
                for session_list in session_lists:
                    df_tmp = df[df['session'].isin(session_list)]
                    df_tmp = get_candidates(df_tmp, co, prefix, treshhold=2, treshhold_topn=treshhold_topn)
                    out.append(df_tmp)
                df = cudf.concat(out)
                del out
                gc.collect()
                df['session'] = df['session'].astype('int32')
                df['cand'] = df['cand'].astype('int32')
                df.to_parquet('./data_folds/fold_' + str(igfold) + '/candidates/' + dataset + '/' + prefix + '_' + str(treshhold_topn) + '/cand.parquet')
                del df
                gc.collect()

train 20
sub 20
